- Inaya Rahmanisa
- 2106708330

For Programming Assignment 3, you are asked to perform re-ranking with learning-to-rank model on the given dataset. In order to do so, you need to preprocess the dataset, index the document collection, ideate the re-ranking features, train the model, and evaluate the performance for further analysis.

PyTerrier is a helpful tool to carry out IR operations. Therefore, we provide a PyTerrier template for this assignment. More information and example can be referred from the previous [Tutorial 1](https://colab.research.google.com/drive/10NpHgtotDQv6bQXEv3CmqyRVQ0BXknlC?usp=sharing) and PyTerrier documentation.


---


*   Fill in the `✋TODO` section
*   Provide justification for your choices
*   Report your experiment results and analysis


---


Happy exploring!

# Setup

In [ ]:
!pip install python-terrier -q
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git -q
!pip install datasets

You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [308]:
!pip show python-terrier

Name: python-terrier
Version: 0.11.0
Summary: Terrier IR platform Python API
Home-page: https://github.com/terrier-org/pyterrier
Author: Craig Macdonald
Author-email: craigm@dcs.gla.ac.uk
License: UNKNOWN
Location: c:\users\inaya\appdata\local\programs\python\python310\lib\site-packages
Requires: dill, tqdm, statsmodels, more-itertools, requests, ir-datasets, pytrec-eval-terrier, wget, pyjnius, deprecated, ir-measures, numpy, jinja2, matchpy, chest, pandas, joblib, scipy
Required-by: pyterrier-t5


In [1]:
import re
import gdown
import pyterrier as pt
import pandas as pd
import numpy as np

from datasets import load_dataset
from pyterrier.measures import *

if not pt.started():
  pt.init(version='snapshot')

c:\Users\inaya\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\inaya\AppData\Local\Temp\ipykernel_6896\3070504568.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: jitpack 2024-09-24 17:43), helper_version=0.0.8]
C:\Users\inaya\AppData\Local\Temp\ipykernel_6896\3070504568.py:11: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
The following code will have the same effect:
pt.terrier.set_version('snapshot')
pt.java.init() # optional, forces java initialisation
  pt.init(version='snapshot')


# Data Preparation

In [2]:

dataset = pt.get_dataset('irds:cord19/trec-covid/round1')

itera = (dataset.get_corpus_iter())
# for i in itera:
#     print(i)

cord19/trec-covid/round1 documents:   0%|          | 0/51078 [00:00<?, ?it/s]

In [3]:
itera = (dataset.get_corpus_iter())
for i in itera:
    print(i.keys())
    break
# print(itera)

cord19/trec-covid/round1 documents:   0%|          | 0/51078 [00:00<?, ?it/s]

dict_keys(['title', 'doi', 'date', 'abstract', 'docno'])


In [4]:
collections = pd.DataFrame(dataset.get_corpus_iter())
queries = pd.DataFrame(dataset.get_topics())

qrels_df = pd.DataFrame(dataset.get_qrels())

cord19/trec-covid/round1 documents: 100%|██████████| 51078/51078 [00:00<00:00, 142955.67it/s]


There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


In [5]:
final_qrels = qrels_df.drop(columns="iteration", axis=1)
duplicate_count = final_qrels.duplicated(subset=["qid", "docno"]).sum()

print(f"Number of duplicated qid-docno pairs: {duplicate_count}")


Number of duplicated qid-docno pairs: 0


In [6]:
from sklearn.model_selection import train_test_split
unique_qids = final_qrels["qid"].unique()


# Split qids into train, dev, and test sets
train_qids, temp_qids = train_test_split(unique_qids, test_size=0.4, random_state=42)
dev_qids, test_qids = train_test_split(temp_qids, test_size=0.5, random_state=42)

# Filter qrels based on splits


train_qrels = final_qrels[final_qrels["qid"].isin(train_qids)]
dev_qrels = final_qrels[final_qrels["qid"].isin(dev_qids)]
test_qrels = final_qrels[final_qrels["qid"].isin(test_qids)]

# Print sizes of each split
print(f"Train set: {len(train_qrels)} rows")
print(f"Dev set: {len(dev_qrels)} rows")
print(f"Test set: {len(test_qrels)} rows")

Train set: 5365 rows
Dev set: 1674 rows
Test set: 1652 rows


In [7]:
collections.head()

,title,doi,date,abstract,docno
0,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,2003-01-13,"BACKGROUND: Rhinovirus, the most common cause ...",xqhn0vbp
1,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,2003-04-28,Recent analyses of human pathogens have reveal...,gi6uaa83
2,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,2003-06-27,"The army of the men of death, in John Bunyan's...",le0ogx1s
3,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,2003-09-12,BACKGROUND: The human leukocyte antigen (HLA) ...,fy4w7xz8
4,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,2003-09-10,BACKGROUND: An epidemic of a Severe Acute Resp...,0qaoam29


In [8]:
queries.head()

,qid,title,description,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


In [9]:
train_qrels.head()

,qid,docno,label
323,2,01yc7lzk,0
324,2,084o1dmp,0
325,2,0eu2ikwa,0
326,2,0gxxuyln,0
327,2,0m5umout,0


## ✋ TODO

In [10]:
import unicodedata
import re

def remove_nonalphanum(text):
  pattern = re.compile('[\W_]+')
  return pattern.sub(' ', text)

def normalize_to_english(text): 
    text = text.lower()

    # normalize to decompose characters (e.g., "â" -> "â")
    normalized_text = unicodedata.normalize('NFD', text)
    # remove diacritic marks using regex (e.g., "â" -> "a")
    normalized_text = re.sub(r'[\u0300-\u036f]', '', normalized_text)
    return normalized_text

In [11]:
# TODO: preprocess data according to needs

collections['title'] = collections['title'].apply(lambda x: normalize_to_english(remove_nonalphanum(x)))
collections['abstract'] = collections['abstract'].apply(lambda x: normalize_to_english(remove_nonalphanum(x)))
collections.drop_duplicates(inplace=True)
collections = collections.drop_duplicates(subset=["docno"], keep="last")


collections = collections.rename(columns={'title': 'text'})

queries['description'] = queries['description'].apply(lambda x: normalize_to_english(remove_nonalphanum(x)))
queries.drop(columns=["title", "narrative"], inplace=True)

In [12]:
# preprocess queries
queries = queries.sort_values(by='qid').reset_index(drop=True)
queries

,qid,description
0,1,what is the origin of covid 19
1,10,has social distancing had an impact on slowing...
2,11,what are the guidelines for triaging patients ...
3,12,what are best practices in hospitals and at ho...
4,13,what are the transmission routes of coronavirus
5,14,what evidence is there related to covid 19 sup...
6,15,how long can the coronavirus live outside the ...
7,16,how long does coronavirus remain stable on sur...
8,17,are there any clinical trials available for th...
9,18,what are the best masks for preventing infecti...


In [13]:
# preprocess train_qrels
train_qrels = train_qrels.sort_values(by='qid').reset_index(drop=True)

# preprocess dev_qrels
dev_qrels = dev_qrels.sort_values(by='qid').reset_index(drop=True)

# preprocess test_qrels
test_qrels = test_qrels.sort_values(by='qid').reset_index(drop=True)

Mengecek distribusi label pada qrels

In [14]:
import pandas as pd

# Function to down-sample the majority class
def downsample_majority_class(df, label_column='label'):
    # Find the minimum number of samples in the non-majority classes
    min_class_size = df[label_column].value_counts().min()
    
    # Down-sample the majority class to match the smallest class size
    df_majority = df[df[label_column] == 0]
    df_minority = df[df[label_column] != 0]
    
    # Randomly sample from the majority class to match the smallest class size
    df_majority_downsampled = df_majority.sample(min_class_size, random_state=42)
    
    # Combine the downsampled majority class with the minority classes
    balanced_df = pd.concat([df_majority_downsampled, df_minority])
    
    return balanced_df

# Apply down-sampling on train, dev, and test sets
train_qrels_balanced = downsample_majority_class(train_qrels)
dev_qrels_balanced = downsample_majority_class(dev_qrels)
test_qrels_balanced = downsample_majority_class(test_qrels)

# Check the value counts after balancing
print(train_qrels_balanced['label'].value_counts())
print(dev_qrels_balanced['label'].value_counts())
print(test_qrels_balanced['label'].value_counts())


label
2    714
0    651
1    651
Name: count, dtype: int64
label
1    229
0    224
2    224
Name: count, dtype: int64
label
2    299
0    235
1    235
Name: count, dtype: int64


In [15]:
print(train_qrels_balanced['label'].value_counts())
print(dev_qrels_balanced['label'].value_counts())
print(test_qrels_balanced['label'].value_counts())

label
2    714
0    651
1    651
Name: count, dtype: int64
label
1    229
0    224
2    224
Name: count, dtype: int64
label
2    299
0    235
1    235
Name: count, dtype: int64


In [16]:
# train_topics = queries[queries['qid'].isin(train_qrels['qid'])]
# dev_topics = queries[queries['qid'].isin(dev_qrels['qid'])]
# test_topics = queries[queries['qid'].isin(test_qrels['qid'])]

train_topics = queries[queries['qid'].isin(train_qrels_balanced['qid'])]
dev_topics = queries[queries['qid'].isin(dev_qrels_balanced['qid'])]
test_topics = queries[queries['qid'].isin(test_qrels_balanced['qid'])]


#  get the first x qids (queries)
# def first_x_qids(topic_set, max_query):
#     first_100_qids = topic_set.drop_duplicates(subset='qid').head(max_query)
#     serp_qrels_first_100_qids = topic_set[topic_set['qid'].isin(first_100_qids['qid'])]
#     return serp_qrels_first_100_qids


# # get first 50 qids for topics and qrels
# train_topics = first_x_qids(train_topics, 50)
# dev_topics = first_x_qids(dev_topics, 50)
# test_topics = first_x_qids(test_topics, 50)
# test_topics = test_topics.rename(columns={'text': 'query'})
# dev_topics = dev_topics.rename(columns={'text': 'query'})
# train_topics = train_topics.rename(columns={'text': 'query'})


# train_qrels= train_qrels[:50]
# dev_qrels = dev_qrels[:50]
# test_qrels = test_qrels[:50]

In [17]:
# train_qrels= train_qrels[:50]
# dev_qrels = dev_qrels[:50]
# test_qrels = test_qrels[:50]

In [18]:
train_topics.shape
# dev_topics.shape
# test_topics.shape

(18, 2)

In [19]:
collections.head()

,text,doi,date,abstract,docno
0,airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,2003-01-13,background rhinovirus the most common cause of...,xqhn0vbp
1,discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,2003-04-28,recent analyses of human pathogens have reveal...,gi6uaa83
2,a new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,2003-06-27,the army of the men of death in john bunyan s ...,le0ogx1s
3,association of hla class i with severe acute r...,10.1186/1471-2350-4-9,2003-09-12,background the human leukocyte antigen hla sys...,fy4w7xz8
4,a double epidemic model for the sars propagation,10.1186/1471-2334-3-19,2003-09-10,background an epidemic of a severe acute respi...,0qaoam29


In [20]:

queries.head()

,qid,description
0,1,what is the origin of covid 19
1,10,has social distancing had an impact on slowing...
2,11,what are the guidelines for triaging patients ...
3,12,what are best practices in hospitals and at ho...
4,13,what are the transmission routes of coronavirus


In [21]:
train_qrels.head()

,qid,docno,label
0,11,0agldesf,1
1,11,oyid5haj,0
2,11,owv52rjk,0
3,11,odmnvv7m,0
4,11,odl1nv4i,0


# Indexing

## ✋ TODO

In [25]:
# TODO: adjust indexer
import os
import shutil


index_path = os.path.join(os.getcwd(), "index")  # Use current working directory + index


if os.path.exists(index_path):
    index_ref = index_path
    # If it exists, remove the directory and all its contents
else:

    # index_path = "./index"
    pd_indexer = pt.IterDictIndexer(index_path, \
                            type = pt.index.IndexingType(1), \
                            tokeniser = pt.index.TerrierTokeniser('utf'), \
                            stemmer = pt.index.TerrierStemmer('porter'), \
                            stopwords = pt.index.TerrierStopwords('terrier'), \
                            blocks = True, \
                            verbose = True,
                            meta={'docno': 20, 'text':4096 , 'doi':4096, 'date':1080, 'abstract': 10800})
    index_ref = pd_indexer.index(collections.to_dict(orient="records"))



In [26]:
index_fact = pt.IndexFactory.of(index_ref)
print(index_fact.getCollectionStatistics().toString())
print(index_fact.getMetaIndex().getKeys())

Number of documents: 51045
Number of terms: 25000
Number of postings: 473366
Number of fields: 1
Number of tokens: 484664
Field names: [text]
Positions:   true

['docno', 'text', 'doi', 'date', 'abstract']


In [27]:
print(index_fact.getMetaIndex().getItem("docno", 10))
# lihat lexicon
for i, (term, le) in enumerate(index_fact.getLexicon()):
  if i % 300 == 0:
    # limit output
    print(term, le.getFrequency())

5s6acr7m
0 104
1910 1
30th 3
654 1
abram 1
aefi 1
alphacoronaviru 26
annulata 2
arau 1
atopi 6
bafilomycin 2
bet 1
bl 1
bronz 1
camouflag 3
ccl22 1
chemiluminesc 8
clandestin 1
commerci 104
cordatum 1
crystallograph 22
daphnia 1
deploi 7
dimension 45
dp 2
edicion 1
endocrinologist 1
eras 1
expans 49
fern 1
focal 7
g15p 1
germani 53
gp350 2
haemagglutinin 14
hemolymphat 1
hmbpp 2
hydrocarbon 8
iga 24
inconsist 1
intercurr 2
isoflavon 3
kaa 1
l1 12
lentivir 28
longev 7
mainten 27
mdea 1
mhc 65
mlb1 1
mucolipin 1
nairoviru 6
neu5gc 1
noncontact 2
nunavut 1
operophtera 1
oxidas 7
paramyxovirina 1
pepitop 1
photoantimicrobi 1
pluripot 9
potenzi 1
procyanidin 3
publishedin 1
radiographi 9
reg 3
revista 1
rotterdam 1
sarcopt 1
sen 1
siderocalin 1
soil 5
stai 17
subtilas 1
t10b9 1
tetramethylen 1
tobramycin 2
triangl 1
ube3a 1
uptak 31
vergleich 1
vp13 1
woefulli 1
zorgelijk 1


In [165]:
di = index_fact.getDirectIndex()
doi = index_fact.getDocumentIndex()
lex = index_fact.getLexicon()

for posting in di.getPostings(doi.getDocumentEntry(0)):
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    print(f"Term ID: {termid}, Term: {lee.getKey()}, Frequency: {posting.getFrequency()}")

Term ID: 0, Term: rhinoviru, Frequency: 1
Term ID: 1, Term: detect, Frequency: 2
Term ID: 2, Term: irradi, Frequency: 1
Term ID: 3, Term: airborn, Frequency: 1
Term ID: 4, Term: ultraviolet, Frequency: 1
Term ID: 5, Term: assai, Frequency: 1
Term ID: 6, Term: effect, Frequency: 1
Term ID: 7, Term: rt, Frequency: 1
Term ID: 8, Term: semi, Frequency: 1
Term ID: 9, Term: pcr, Frequency: 1
Term ID: 10, Term: nest, Frequency: 1


# Baseline

In [28]:
bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25")

C:\Users\inaya\AppData\Local\Temp\ipykernel_6896\3225921946.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25")


In [29]:
test_topics = test_topics.rename(columns={'description': 'query'})
dev_topics = dev_topics.rename(columns={'description': 'query'})
train_topics = train_topics.rename(columns={'description': 'query'})

In [293]:
eval_baseline = pt.Experiment(
    [bm25], # models
    test_topics,
    test_qrels_balanced,
    eval_metrics=[P@1, P@5, P@10, "map", nDCG@10],
    names=["BM25"], # for model's name readability
)
eval_baseline


,name,P@1,P@5,P@10,map,nDCG@10
0,BM25,0.833333,0.733333,0.55,0.176064,0.535262


In [193]:
# eval_baseline = pt.Experiment(
#     [bm25], # models
#     test_topics,
#     test_qrels, # qrels
#     eval_metrics=[P@1, P@5, P@10, "map", nDCG@10],
#     names=["BM25"], # for model's name readability
# )
# eval_baseline


,name,P@1,P@5,P@10,map,nDCG@10
0,BM25,0.833333,0.733333,0.55,0.176064,0.535262


In [194]:
serp = bm25.transform(test_topics)
serp

,qid,docid,docno,rank,score,query
0,1,29980,gv1k7u7j,0,19.805728,what is the origin of covid 19
1,1,44319,1mjaycee,1,17.981557,what is the origin of covid 19
2,1,30921,yn8nzezq,2,16.465071,what is the origin of covid 19
3,1,33536,h2uc7ria,3,15.798867,what is the origin of covid 19
4,1,48128,msohf5oa,4,15.184478,what is the origin of covid 19
...,...,...,...,...,...,...
5995,5,18414,v0zcbdhk,995,7.464044,what drugs have been active against sars cov o...
5996,5,18275,sojsk9y7,996,7.422614,what drugs have been active against sars cov o...
5997,5,42388,kp7qk49p,997,7.422614,what drugs have been active against sars cov o...
5998,5,45327,u9wmzvkz,998,7.422614,what drugs have been active against sars cov o...


# Re-rank

## Feature

### ✋ TODO

In [30]:
# TODO: propose and implement features

# Example:
def jaccard(doc, query):
    doc_set = set(doc.split())
    query_set = set(query.split())
    
    intersection = len(doc_set & query_set)
    union = len(doc_set | query_set)
    
    return intersection / union if union != 0 else 0.0

def other_feature(doc, query): # cosine similarity
    doc_tokens = doc.split()
    query_tokens = query.split()
    
    vocabulary = list(set(doc_tokens + query_tokens))
    
    doc_vector = np.array([doc_tokens.count(word) for word in vocabulary])
    query_vector = np.array([query_tokens.count(word) for word in vocabulary])
    
    dot_product = np.dot(doc_vector, query_vector)
    
    doc_magnitude = np.sqrt(np.dot(doc_vector, doc_vector))
    query_magnitude = np.sqrt(np.dot(query_vector, query_vector))
    
    if doc_magnitude == 0 or query_magnitude == 0:
        return 0.0  # handle edge case where one of the vectors has zero magnitude
    cosine_similarity = dot_product / (doc_magnitude * query_magnitude)
    
    return cosine_similarity

def generate_features(doc, query):
    return np.array([jaccard(doc, query), \
                     other_feature(doc, query)])

In [31]:
features = pt.apply.doc_features(lambda row: generate_features(row["text"], row["query"]))

In [32]:
K = 30

In [33]:
# Get "text" column using `pt.text.get_text(index_ref, "text")`
pipeline = (bm25 % K) >> pt.text.get_text(index_ref, "text") >> (features ** bm25)

## Learning-to-Rank

### ✋ TODO

In [34]:
!pip install xgboost-cpu



You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [35]:
# TODO: perform hyperparameter tuning on the LambdaMART
import xgboost as xgb

# Below hyperparameters are taken from the example on PyTerrier documentation:
# https://pyterrier.readthedocs.io/en/latest/ltr.html#gradient-boosted-trees-lambdamart

# XGBoost hyperparameters documentation:
# https://xgboost.readthedocs.io/en/stable/parameter.html

lmart_model = xgb.sklearn.XGBRanker(objective = 'rank:ndcg',
                              learning_rate = 0.1,
                              gamma = 1.0,
                              min_child_weight = 0.1,
                              max_depth = 6,
                              verbose = 2,
                              random_state = 42)

Hyperparameter Tuning

In [36]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [0, 0.5, 1.0],
    'min_child_weight': [0.1, 0.5, 1.0],
    'max_depth': [3, 6, 9],
    'subsample': [0.5, 0.7, 0.8],
}

In [37]:
import itertools
from tqdm import tqdm

# Flatten the parameter grid into a list of dictionaries
param_combinations = [
    dict(zip(param_grid.keys(), values))
    for values in itertools.product(*param_grid.values())
]

best_score = -float('inf')  # init to negative infinity
best_params = None

for params in tqdm(param_combinations, desc="Hyperparameter Tuning"):
    
    lmart_model = xgb.sklearn.XGBRanker(objective='rank:ndcg', random_state=42, **params)
    
   
    lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form="ltr")
    lmart_pipe.fit(train_topics, train_qrels_balanced, dev_topics, dev_qrels_balanced)
    # lmart_pipe.fit(train_topics, train_qrels, dev_topics, dev_qrels)


    results = lmart_pipe(dev_topics)
    score = pt.Utils.evaluate(results, dev_qrels_balanced,  metrics=['ndcg'])['ndcg']

    # score = pt.Utils.evaluate(results, dev_qrels, metrics=['ndcg'])['ndcg']

    if score > best_score:
        best_score = score
        best_params = params

print(f"Best score: {best_score}")
print(f"Best hyperparameters: {best_params}")


Hyperparameter Tuning:   0%|          | 0/243 [00:00<?, ?it/s]C:\Users\inaya\AppData\Local\Temp\ipykernel_6896\492794354.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  score = pt.Utils.evaluate(results, dev_qrels_balanced,  metrics=['ndcg'])['ndcg']
Hyperparameter Tuning:   0%|          | 1/243 [00:04<18:16,  4.53s/it]C:\Users\inaya\AppData\Local\Temp\ipykernel_6896\492794354.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  score = pt.Utils.evaluate(results, dev_qrels_balanced,  metrics=['ndcg'])['ndcg']
Hyperparameter Tuning:   1%|          | 2/243 [00:08<16:38,  4.14s/it]C:\Users\inaya\AppData\Local\Temp\ipykernel_6896\492794354.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  score = pt.Utils.evaluate

Best score: 0.16728927834970073
Best hyperparameters: {'learning_rate': 0.05, 'gamma': 0.5, 'min_child_weight': 1.0, 'max_depth': 3, 'subsample': 0.5}


# Evaluation


## ✋ TODO

In [38]:
lmart_model = xgb.sklearn.XGBRanker(objective = 'rank:ndcg',    
                              verbose = 2,
                              random_state = 42,
                              **best_params)

In [39]:
lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form = "ltr")
lmart_pipe.fit(train_topics, train_qrels_balanced, dev_topics, dev_qrels_balanced)

c:\Users\inaya\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [07:36:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-02e25fbcaf14bc601-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


In [61]:
bm25_full = pt.BatchRetrieve(index_ref, wmodel="BM25", fields=["text", "abstract"])


C:\Users\inaya\AppData\Local\Temp\ipykernel_6896\2009184872.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25_full = pt.BatchRetrieve(index_ref, wmodel="BM25", fields=["text", "abstract"])


In [58]:
import pickle
import gzip

with gzip.open(f"lmart_pipe.pkl", "wb") as f:
        pickle.dump(
             lmart_pipe
        , f)

TypeError: no default __reduce__ due to non-trivial __cinit__

In [ ]:
# lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form = "ltr")
# lmart_pipe.fit(train_topics, train_qrels, dev_topics, dev_qrels)

c:\Users\inaya\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [13:24:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-02e25fbcaf14bc601-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


In [296]:
# evaluate on dev set
dev_pred = lmart_pipe.transform(dev_topics)

eval_results = pt.Utils.evaluate(
    dev_pred,     
    dev_qrels_balanced   
)

print(eval_results)


{'map': 0.04075017155575187, 'ndcg': 0.151036477387153}


C:\Users\inaya\AppData\Local\Temp\ipykernel_2912\694728764.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval_results = pt.Utils.evaluate(


In [41]:
eval_results = pt.Experiment([ bm25, lmart_pipe], \
                            test_topics, \
                            test_qrels, \
                            eval_metrics=[P@10, "map", "recip_rank", nDCG@10], \
                            names=["BM25", "L2R"], \
                            baseline=0) # statistical significance test

In [42]:
eval_results

,name,map,recip_rank,P@10,nDCG@10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P@10 +,P@10 -,P@10 p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.176064,0.888889,0.55,0.535262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L2R,0.117786,0.783333,0.45,0.409725,0.0,6.0,0.029809,1.0,1.0,0.489336,1.0,3.0,0.295559,1.0,5.0,0.270139


In [43]:
!pip install -U sentence-transformers -q

You should consider upgrading via the 'C:\Users\inaya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [44]:
from sentence_transformers import SentenceTransformer, CrossEncoder
from sentence_transformers.util import cos_sim

In [45]:
# TODO: Instantiate bi-encoder
# Feel free to choose the model
# A few examples of the pretrained model: https://sbert.net/docs/sentence_transformer/pretrained_models.html#original-models
bimodel = SentenceTransformer('all-MiniLM-L6-v2')

def _biencoder_apply(dataframe):
    # TODO: Encode the raw document and raw query using sentence transformer,
    # then compute their similarity score
    # Example: https://pyterrier.readthedocs.io/en/latest/text.html#examples-of-sentence-transformers
    # return 0
    query_embs = bimodel.encode(dataframe['query'].values)
    concatenated_text = dataframe['text'] + " " + dataframe['abstract']
    
    doc_embs = bimodel.encode(concatenated_text.values)
    
    
    scores = cos_sim(query_embs, doc_embs)
    return scores[0]

bi_encT = pt.apply.doc_score(_biencoder_apply, batch_size=128)

In [46]:
collections

,text,doi,date,abstract,docno
0,airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,2003-01-13,background rhinovirus the most common cause of...,xqhn0vbp
1,discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,2003-04-28,recent analyses of human pathogens have reveal...,gi6uaa83
2,a new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,2003-06-27,the army of the men of death in john bunyan s ...,le0ogx1s
3,association of hla class i with severe acute r...,10.1186/1471-2350-4-9,2003-09-12,background the human leukocyte antigen hla sys...,fy4w7xz8
4,a double epidemic model for the sars propagation,10.1186/1471-2334-3-19,2003-09-10,background an epidemic of a severe acute respi...,0qaoam29
...,...,...,...,...,...
51073,prevalence of antibodies against transmissible...,10.1016/j.vas.2018.11.003,2019-06-30,abstract transmissible gastroenteritis tge is ...,4xhcamks
51074,can we increase public awareness without creat...,10.1016/j.pec.2013.10.023,2014-02-28,,z2rqz3jx
51075,fast degrading polyesters as sirna nano carrie...,10.1016/j.jconrel.2008.06.010,2008-12-18,abstract a potential sirna carrier for pulmona...,wncuc903
51076,eif3f a central regulator of the antagonism at...,10.1016/j.biocel.2013.06.001,2013-10-31,abstract the eukaryotic initiation factor 3 su...,4x1ckr2i


In [47]:
CUT_OFF = 30

# bm25_biencoder = (bm25 % CUT_OFF) >> bi_encT
# bm25_biencoder = (bm25 % CUT_OFF) >> pt.text.get_text(index_ref,"text", "abstract") >> bi_encT


In [48]:
# TODO: Instantiate cross-encoder
# Feel free to choose the model
# A few examples of the pretrained model: https://sbert.net/docs/cross_encoder/pretrained_models.html
crossmodel = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

def _crossencoder_apply(dataframe):
    # TODO: Use cross-encoder to directly predict the similarity score of the raw document and raw query
    # Example: https://pyterrier.readthedocs.io/en/latest/text.html#examples-of-sentence-transformers
    concatenated_text = dataframe['text'] + " " + dataframe['abstract']
    return crossmodel.predict(list(zip(dataframe['query'].values, concatenated_text.values)))

cross_encT = pt.apply.doc_score(_crossencoder_apply, batch_size=128)

In [66]:
# bm25_crossencoder = (bm25 % CUT_OFF) >> cross_encT
bm25_pipe = (bm25 % K) >> pt.text.get_text(index_ref, "text") >> pt.text.get_text(index_ref, "abstract")

In [49]:
biencoder_crossencoder = pt.terrier.Retriever(index_ref,metadata=["docno", "text", "abstract"]) >> (bi_encT % CUT_OFF) >> cross_encT


In [50]:

# TODO: Perform experiment for every model
# (BM25, BM25 >> Bi-encoder, BM25 >> Cross-encoder, and Bi-encoder >> Cross-encoder),
# with significance testing, where the baseline is BM25
# Example: https://pyterrier.readthedocs.io/en/latest/experiments.html#significance-testing
eval_metrics=[P@10, "map", "recip_rank", nDCG@10]
eval_results = pt.Experiment([ bm25, biencoder_crossencoder, lmart_pipe],\
                           test_topics, \
                            test_qrels, \
                            eval_metrics=eval_metrics, \
                            names=["BM25",  "Biencoder CrossEncoder", "L2R"], \
                             filter_by_qrels=True,\
                            baseline=0) # statistical significance test



pt.apply: 100%|██████████| 2/2 [00:20<00:00, 10.02s/row]


In [51]:
eval_results

,name,map,recip_rank,P@10,nDCG@10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P@10 +,P@10 -,P@10 p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.176064,0.888889,0.550000,0.535262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Biencoder CrossEncoder,0.171005,1.000000,0.633333,0.633744,1.0,5.0,0.926699,1.0,0.0,0.363217,3.0,2.0,0.507001,3.0,3.0,0.404862
2,L2R,0.117786,0.783333,0.450000,0.409725,0.0,6.0,0.029809,1.0,1.0,0.489336,1.0,3.0,0.295559,1.0,5.0,0.270139


In [311]:
import pickle



def savemodels(model, model_filename):
    # Save models and results
    with open(f"{model_filename}.pkl", "wb") as f:
        # Save a dictionary containing models and eval_results
        pickle.dump(
             model
        , f)

savemodels(biencoder_crossencoder, 'biencoder_crossencoder')

In [305]:

lmart_pipe
with open('model.pkl','wb') as f:
    pickle.dump(bm25, f, protocol=pickle.HIGHEST_PROTOCOL)

In [312]:
savemodels(bm25, "bm25")

In [ ]:
savemodels(lmart_pipe, 'lmart_pipe')

In [302]:
savemodels(bm25, 'bm25')

In [54]:
query = "covid-19"

query = normalize_to_english(remove_nonalphanum(query))
queries_df = pd.DataFrame([{"qid": "1", "query": query}])

In [307]:
!pip show pyterrier


In [67]:
bm25_pipe.transform(queries_df).sort_values(by=['rank'], ascending=[ True])

,qid,docid,docno,rank,score,query,text,abstract
0,1,26284,5l7516xp,0,12.415991,covid 19,where are we now with covid 19,as a practicing front line clinician in the ac...
1,1,29800,8m3wg1tg,1,12.415991,covid 19,o anestesiologista e a covid 19,
2,1,29872,450z0tv1,2,12.415991,covid 19,the anesthesiologist and covid 19,
3,1,29902,s7idehep,3,12.415991,covid 19,medicine before covid 19 and after,
4,1,30314,e5ya4vio,4,12.415991,covid 19,covid 19 escalates,who calls for more testing as countries worldw...
5,1,30331,j1q59u9c,5,12.415991,covid 19,beyond the science of covid 19,models of the new coronavirus s spread are imp...
6,1,32248,3gbxbg5f,6,12.415991,covid 19,covid 19 and interconnectedness,
7,1,34339,w11wop27,7,12.415991,covid 19,diagnosing covid 19,hina continues to fight the outbreak of a nove...
8,1,34406,6duc06et,8,12.415991,covid 19,diabetes and covid 19,we acknowledge the frontline medical staff wor...
9,1,34433,e4hmo4yc,9,12.415991,covid 19,the covid 19 epidemic,abstract the current outbreak of the novel cor...


In [72]:

# with open('model.pkl','wb') as f:
#     pickle.dump(bm25_pipe, f, protocol=pickle.HIGHEST_PROTOCOL)
import joblib
with open('model.joblib', 'wb') as f:
    joblib.dump(bm25_pipe, f)

TypeError: no default __reduce__ due to non-trivial __cinit__

In [55]:
biencoder_crossencoder.transform(queries_df).sort_values(by=['rank'], ascending=[ True])

pt.apply: 100%|██████████| 1/1 [00:00<00:00,  5.17row/s]


,qid,docid,docno,text,abstract,score,query,rank
307,1,31257,dvz2lg9e,covid 19 is a paradigm change to be expected i...,,9.181604,covid 19,0
715,1,26230,ur8i1pp8,covid 19 a novel coronavirus and a novel chall...,,8.838291,covid 19,1
230,1,40904,nck4f5ny,covid 19 a critical care perspective informed ...,,8.548873,covid 19,2
37,1,22868,n8e2n30b,covid 19 a novel zoonotic disease caused by a ...,,8.220176,covid 19,3
748,1,30636,6nju1s0w,covid 19 a challenge for emergency medicine an...,,8.175814,covid 19,4
913,1,37588,1jeh98p7,covid 19 another infectious disease emerging a...,,8.107410,covid 19,5
721,1,26349,4ki951ls,coronavirus disease covid 19 a new threat to p...,,7.852543,covid 19,6
541,1,31787,g9j2enbm,covid 19 con afectacion pulmonar una enfermeda...,,7.576957,covid 19,7
82,1,31083,qczuydvb,covid 19 illness in native and immunosuppresse...,,7.327579,covid 19,8
457,1,44377,6ye0x26q,the global community needs to swiftly ramp up ...,,7.232357,covid 19,9


In [272]:
collections.iloc[12]

text        towards evidence based gis driven national spa...
doi                                     10.1186/1476-072x-3-1
date                                               2004-01-28
abstract    the term geographic information systems gis ha...
docno                                                ri5v6u4x
Name: 12, dtype: object

In [56]:
bm25.transform(queries_df).sort_values(by=['rank'], ascending=[ True])[:50]

,qid,docid,docno,rank,score,query
0,1,26284,5l7516xp,0,12.415991,covid 19
1,1,29800,8m3wg1tg,1,12.415991,covid 19
2,1,29872,450z0tv1,2,12.415991,covid 19
3,1,29902,s7idehep,3,12.415991,covid 19
4,1,30314,e5ya4vio,4,12.415991,covid 19
5,1,30331,j1q59u9c,5,12.415991,covid 19
6,1,32248,3gbxbg5f,6,12.415991,covid 19
7,1,34339,w11wop27,7,12.415991,covid 19
8,1,34406,6duc06et,8,12.415991,covid 19
9,1,34433,e4hmo4yc,9,12.415991,covid 19


In [57]:
# bm25.transform(queries_df)
lmart_pipe.transform(queries_df).sort_values(by=['rank'], ascending=[ True])

,qid,docid,docno,score,query,text,features,rank
21,1,26300,3ztfw8lj,-0.181927,covid 19,covid 19 are we getting the communications right,"[0.25, 0.4999999999999999, 11.715107641920778]",0
23,1,29696,yuz510tj,-0.181927,covid 19,covid 19 in endoscopy time to do more,"[0.25, 0.4999999999999999, 11.715107641920778]",1
14,1,14693,7mov6iyk,-0.207623,covid 19,covid 19 in der allgemeinarztpraxis,"[0.4, 0.6324555320336759, 11.715107641920778]",2
15,1,16026,8qzqjtw7,-0.207623,covid 19,nano research for covid 19,"[0.4, 0.6324555320336759, 11.715107641920778]",3
16,1,25032,okjn6hgv,-0.207623,covid 19,covid 19 in der praxis,"[0.4, 0.6324555320336759, 11.715107641920778]",4
17,1,25634,600e9w29,-0.207623,covid 19,offline covid 19 a reckoning,"[0.4, 0.6324555320336759, 11.715107641920778]",5
19,1,26014,wgob5qf6,-0.207623,covid 19,covid 19 in pregnant women,"[0.4, 0.6324555320336759, 11.715107641920778]",6
20,1,26297,b358eb10,-0.207623,covid 19,preventing a covid 19 pandemic,"[0.4, 0.6324555320336759, 11.715107641920778]",7
26,1,29901,n5xlpqms,-0.207623,covid 19,dementia care during covid 19,"[0.4, 0.6324555320336759, 11.715107641920778]",8
27,1,30144,a60bp1gm,-0.207623,covid 19,commentary covid 19 and diabetes,"[0.4, 0.6324555320336759, 11.715107641920778]",9


In [75]:
index_path = os.path.join(os.getcwd(), "index")
print(index_path)
index_ref = pt.IndexFactory.of(index_path)

c:\Users\inaya\Documents\LOCAL_TERM7\LOCAL_IR\tp4-ir\tp4\search\index


In [82]:
corpus_iter = index_ref.get_corpus_iter()

# Creating a dictionary to store the docno, text, and abstract
corpus_dict = {}
texts = []
abstracts = []



collections.to_csv('collections.csv', index=False) 

# pt.text.get_text(index_ref, "text")
# for i in 
    # docno = doc['docno']  # Assuming each document has 'docno', 'text', and 'abstract'
    # if docno in docnos.values:  # Only get documents that are in the docnos list
    #     print(docno)
    #     print(doc.)

In [84]:
docnos = ['le0ogx1s']

In [86]:
# df = pd.read_csv('collections.csv')
collections[collections['docno'].isin(docnos)]
# Display the DataFrame
# df

,text,doi,date,abstract,docno
2,a new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,2003-06-27,the army of the men of death in john bunyan s ...,le0ogx1s


In [87]:
collections2 = pd.DataFrame(dataset.get_corpus_iter())
collections2.to_csv('collections.csv', index=False) 


cord19/trec-covid/round1 documents: 100%|██████████| 51078/51078 [00:00<00:00, 144518.97it/s]


In [88]:
stemmer = pt.index.TerrierStemmer('porter')
stemmer.stem("cooling")

'cool'

In [89]:
lists = ['cooling', 'cool', 'cooles', 'and']

In [94]:
stopwords = pt.index.TerrierStopwords('terrier')
# print([ i for i in lists if i not in stopwords])
# stopwords
# for i in stopwords:
#     print(i)
print(stopwords._to_obj)

<function TerrierStopwords._to_obj at 0x00000241A4E8F9A0>


In [95]:
word = "the"
if stopwords.is_stopword(word):
    print(f"'{word}' is a stopword.")

AttributeError: 'TerrierStopwords' object has no attribute 'is_stopword'

In [ ]:
stopwords_in_index = index_ref.getProperties().get("pyterrier.stopwords")  # For custom stopwords
print(stopwords_in_index)

AttributeError: 'org.terrier.structures.Index' object has no attribute 'get'

In [98]:
print(index_ref.stopwords)

AttributeError: 'org.terrier.structures.Index' object has no attribute 'stopwords'

In [99]:
print(index_ref.getCollectionStatistics().toString())

Number of documents: 51045
Number of terms: 25000
Number of postings: 473366
Number of fields: 1
Number of tokens: 484664
Field names: [text]
Positions:   true



In [110]:
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already done
nltk.download('stopwords')


texx = "i read this morning with cooling fresh air with the best"
# Get the English stopwords list
stop_words = set(stopwords.words('english'))

stemmer = pt.index.TerrierStemmer('porter')
stemmer.stem("cooling")


cleaned = [i for i in texx.split() if i not in stop_words]
cleaned = [stemmer.stem(i) for i in cleaned]
cleaned = " ".join(cleaned)
cleaned = cleaned.strip()
cleaned

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\inaya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'read morn cool fresh air best'